In [14]:
!pip install -q -U git+https://github.com/huggingface/transformers
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q pythainlp fairseq sacremoses

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with 

In [15]:
!nvidia-smi

Sat May  4 08:55:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              36W /  70W |   2637MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.2
Is CUDA enabled? True


In [17]:
import torch
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from pythainlp.translate import Translate
import pandas as pd

In [18]:
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# model = BlipForConditionalGeneration.from_pretrained(
#     "Salesforce/blip-image-captioning-large",
#     device_map='cuda',
#     torch_dtype=torch.float16
# )
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base",
    device_map='cuda',
    torch_dtype=torch.float16
)
translate_model = Translate('en', 'th', use_gpu=True)

In [19]:
def process_image(path: Path, is_coco: bool):
    image = Image.open(path)
    image_id = str(path.relative_to(path.parents[1 if is_coco else 2]).with_suffix(''))
    inputs = processor(image, return_tensors="pt").to("cuda", torch.float16)
    out = model.generate(**inputs) # max_new_tokens=20
    caption_en = processor.decode(out[0], skip_special_tokens=True)
    caption_th = translate_model.translate(caption_en)
    return (image_id, caption_th)

def process_images(paths):
    return [process_image(p, is_coco) for p, is_coco in tqdm(paths)]

In [20]:
if __name__ == '__main__':
    test_path_coco = Path('/kaggle/input/coco-2017-dataset/coco2017/test2017')
    food_path = Path('/kaggle/input/thai-language-image-captioning/test/test/food')
    travel_path = Path('/kaggle/input/thai-language-image-captioning/test/test/travel')
    
    image_paths = [(p, True) for p in test_path_coco.iterdir()] + \
                  [(p, False) for p in food_path.iterdir()] + \
                  [(p, False) for p in travel_path.iterdir()]
    
    image_paths = [(p, True) for p in list(test_path_coco.iterdir())[:30]] + \
                  [(p, False) for p in list(food_path.iterdir())[:30]] + \
                  [(p, False) for p in list(travel_path.iterdir())[:30]]


    results = process_images(image_paths)

    submission = pd.DataFrame(results, columns=['image_id', 'caption'])
    submission.to_csv('submission_blip_base.csv', index=False)


100%|██████████| 90/90 [00:26<00:00,  3.35it/s]
